In [1]:
import requests
import json
import elevenlabs
from openai import OpenAI
import time

In [2]:
with open('../credentials.json', 'r') as file:
    cred = json.load(file)

elevenlabs.set_api_key(cred["elevenlabs_api_key"])
client = OpenAI(api_key=cred["openai_api_key"])

thread_id = "thread_L4ERbnzOKTrnwjZdlPngGMCN"

In [3]:
my_assistant = client.beta.assistants.retrieve("asst_nv9sFR2LXHefrpwws4EUT6xU")
print(my_assistant)

Assistant(id='asst_nv9sFR2LXHefrpwws4EUT6xU', created_at=1700273846, description=None, file_ids=['file-YU140dSobQqTEqGsVmAC7Uu2', 'file-np8tBTt7dR7wYpNk3ohJ8EO8', 'file-FKMsMC9rdprq3KL9UZgaR1LZ', 'file-Bt2zUnsToBjhwFWC8TD5x3Fx'], instructions='You are Alan Watts, the British writer and speaker known for your interpretation of Eastern philosophy. In your responses you will display a heavy influence from Zen literature. \n\nYour responses should always err on the side of brevity to the point of being cryptic. \n\nResponses should not be longer than absolutely necessary. \n\n', metadata={}, model='gpt-3.5-turbo-16k', name='Alan', object='assistant', tools=[ToolCodeInterpreter(type='code_interpreter')])


In [4]:
run = client.beta.threads.retrieve(
    thread_id = thread_id
)

print(run)

Thread(id='thread_L4ERbnzOKTrnwjZdlPngGMCN', created_at=1700274727, metadata={}, object='thread')


In [54]:
audio = elevenlabs.generate(
    text = messages.data[0].content[0].text.value,
    voice = "vDxT1hhO5CjgzUI7e2vF"
)

elevenlabs.play(audio)

In [15]:
def new_question(question_text):
  message_request = client.beta.threads.messages.create(
      thread_id = thread_id,
      role = "user",
      content = question_text
  )

  run = client.beta.threads.runs.create(
    thread_id = message_request.thread_id,
    assistant_id = my_assistant.id
  )

  run = client.beta.threads.runs.retrieve(
    thread_id = message_request.thread_id,
    run_id = run.id
  )

  def wait_for_completed():
      while True:
          response = client.beta.threads.runs.retrieve(thread_id = thread_id, run_id = run.id)
          if response.status == "completed":
              return response
          time.sleep(1)

  run_return = wait_for_completed()

  messages = client.beta.threads.messages.list(
    thread_id = message_request.thread_id
  )

  audio = elevenlabs.generate(
      text = messages.data[0].content[0].text.value,
          voice = "vDxT1hhO5CjgzUI7e2vF"
  )

  elevenlabs.play(audio)

In [17]:
new_question("How to enjoy nature?")